In [64]:

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils

batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
model = Sequential()
model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))# Layer 1
model.add(Activation('relu'))# Layer 2
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))# Layer 3
model.add(Activation('relu'))# Layer 4
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))# Layer 5
model.add(Dropout(0.25))# Layer 6
model.add(Flatten())# Layer 7
model.add(Dense(128))# Layer 8
model.add(Activation('relu'))# Layer 9
model.add(Dropout(0.5))# Layer 10
model.add(Dense(nb_classes))# Layer 11
model.add(Activation('softmax')) # 12
model.compile(loss='categorical_crossentropy', optimizer='adadelta')

for epoch in range(0,nb_epoch):
    model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=1,
          show_accuracy=True, verbose=1, validation_data=(X_test, Y_test))
    # ----------------My code addition-------------------------------
    # Perturb the weights 
    model_dup= model
    score_tmp_ref = model.evaluate(X_train, Y_train, show_accuracy=True, verbose=0)
    max_accuracy = score_tmp_ref[1]
    flag =0
    flag_of_exploration =0
    for layer_num,layer in enumerate(model.layers):# layers.get_weights are lists
        if layer_num == 0:
            weights = model.layers[layer_num].get_weights()[0]# we need only non bias weights so index 0
            # To copy weights into a list
            weights_list_perturbed= weights.tolist()
            biases = model.layers[layer_num].get_weights()[1]
            tmp_shape_counter = np.array(weights).shape
            # Perturbation .. may have to be conditioned on the layer
            for filters in range(0,tmp_shape_counter[0]):
                for input_feature in range(0,tmp_shape_counter[1]):
                    for rows in range(0,tmp_shape_counter[2]):
                        for cols in range(0,tmp_shape_counter[3]):
                            for time in [1,2]:
                                if flag == 0:
                                    weights_list_perturbed[filters][input_feature][rows][cols] = weights[filters][input_feature][rows][cols] +.01
                                    flag =1
                                else:
                                    weights_list_perturbed[filters][input_feature][rows][cols] = weights[filters][input_feature][rows][cols] -.01
                                    flag =0
                                # Set the alternative weights in duplicate model
                                new_weights = []
                                new_weights.append(np.array(weights_list_perturbed))
                                new_weights.append(biases)
                                model_dup.layers[layer_num].set_weights(new_weights)
                                #  To evaluate with perturbed set of weights but on the train set
                                score_tmp = model_dup.evaluate(X_train, Y_train,show_accuracy=True, verbose=0)
                                #  Store the weight for the maximum accuracy from duplicate model
                                print(filters,input_feature,rows,cols)
                                if score_tmp[1] > max_accuracy:
                                    print('yes')
                                    flag_of_exploration =1
                                    max_accuracy = score_tmp[1]
                                    new_weights_by_layer=[]
                                    for lay in model_dup.layers:
                                        new_weights_by_layer.append(lay.get_weights())
    #  Here update the weights if flag is set but in the actual model
    if flag_of_exploration ==1 :
        #  To change the weights of the actual model not duplicate
        for i,l in enumerate(model.layers):
            model.layers[i].set_weights(new_weights_by_layer[i])
    #-----------------------------------addition ends here-------------------------------
    score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)# We need test scores after every update
    print('Test score:', score[0])
    print('Test accuracy:', score[1])
print('training_complete......')


X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 26s - loss: 0.2349 - acc: 0.9284 - val_loss: 0.0554 - val_acc: 0.9832
0 0 0 0
yes
0 0 0 0
0 0 0 1
0 0 0 1
yes
0 0 0 2
0 0 0 2
yes
0 0 1 0
0 0 1 0
yes
0 0 1 1
0 0 1 1
0 0 1 2
0 0 1 2
0 0 2 0
0 0 2 0
0 0 2 1
0 0 2 1
0 0 2 2
0 0 2 2
1 0 0 0
1 0 0 0
1 0 0 1
1 0 0 1
1 0 0 2
1 0 0 2
yes
1 0 1 0
1 0 1 0
1 0 1 1
1 0 1 1
1 0 1 2
1 0 1 2
1 0 2 0
1 0 2 0
1 0 2 1
1 0 2 1
1 0 2 2
1 0 2 2
2 0 0 0
2 0 0 0
2 0 0 1
2 0 0 1
2 0 0 2
2 0 0 2
2 0 1 0
2 0 1 0
2 0 1 1
2 0 1 1
2 0 1 2
2 0 1 2
2 0 2 0
2 0 2 0
2 0 2 1
2 0 2 1
2 0 2 2
2 0 2 2
3 0 0 0
3 0 0 0
3 0 0 1
3 0 0 1
3 0 0 2
3 0 0 2
3 0 1 0
3 0 1 0
3 0 1 1
3 0 1 1
3 0 1 2
3 0 1 2
3 0 2 0
3 0 2 0
3 0 2 1
3 0 2 1
3 0 2 2
yes
3 0 2 2
4 0 0 0
yes
4 0 0 0
4 0 0 1
4 0 0 1
4 0 0 2
4 0 0 2
4 0 1 0
4 0 1 0
4 0 1 1
4 0 1 1
4 0 1 2
4 0 1 2
4 0 2 0
4 0 2 0
4 0 2 1
4 0 2 1
4 0 2 2
4 0 2 2
5 

KeyboardInterrupt: 